In [1]:
import logging

from data_collection.models import Game, Round
from data_collection.questions_candidates import get_question_candidates
from data_collection.questions_candidates_entropy import get_question_candidates_scored
from data_collection.questions_pick import pick_question
from data_collection.semaphore import set_semaphore
from data_collection.guess import guess
from data_collection.answer import answer
from data_collection.simulate_game import run_game


logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

%reload_ext autoreload
%autoreload 2
%autoreload now
from dotenv import load_dotenv
from openai import AsyncOpenAI

from data_collection.utils import call_gpt4, dataset

load_dotenv()

True

In [2]:
set_semaphore("gpt-4", 10)

In [3]:
client = AsyncOpenAI()

In [ ]:
game = await run_game(client, "carrot")

Game carrot, round 0


2024-07-05 22:21:30,730 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-05 22:21:32,971 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-05 22:21:34,902 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-05 22:21:36,209 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-05 22:21:37,642 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Game carrot, round 1


2024-07-05 22:21:39,207 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-05 22:21:41,726 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-05 22:21:43,144 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-05 22:21:44,273 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-05 22:21:45,763 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Game carrot, round 2


In [18]:
import datetime
import json

ds = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
with open(f"dataset_{ds}.json", "w") as f:
    json.dump(dataset, f)

TypeError: Object of type Game is not JSON serializable

In [34]:
response = await call_gpt4(client, "Ask your first question", temperature=0.1)

2024-07-05 18:58:51,087 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [147]:
game = Game(
    rounds=[
        Round(
            question="Is it a living object?",
            answer="yes"
        ),
        Round(
            question="Is it edible?",
            answer="yes"
        ),
        Round(
            question="Is it a fruit?",
            answer="no"
        ),
        Round(
            question="Is it an animal?",
            answer="no"
        ),
        Round(
            question="Is it a dairy product?",
            answer="no"
        ),
        Round(
            question="Is it a vegetable?",
            answer="yes"
        ),
        Round(
            question="Is it commonly green in color?",
            answer="no"
        ),
        Round(
            question="Is it commonly used in salads?",
            answer="no",
        ),
        Round(
            question="Is it commonly red in color?",
            answer="no",
        ),
        Round(
            question="Is it commonly cooked before eating?",
            answer="yes",
        ),
        # Round(
        #     question="Is it commonly used in soups?",
        #     answer="yes",
        #     guess="pumpkin",
        # ),
        Round(
            question="Is it commonly orange in color?",
            answer="yes",
            guess="carrot"
        )
    ]
)

In [136]:
candidates = await get_question_candidates(client, game)

2024-07-05 21:09:04,939 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [137]:
candidates

['Is it commonly used in stews?',
 'Is it a root vegetable?',
 'Is it commonly used in making purees?',
 'Is it commonly used in making baby food?',
 'Is it commonly used in making juices?',
 'Is it a tuber?',
 'Is it commonly used in making pies?',
 'Is it commonly used in making fries?',
 'Is it commonly used in making casseroles?',
 'Is it commonly used in making curries?']

In [138]:
candidates_scored = await get_question_candidates_scored(client, game, candidates)

2024-07-05 21:09:08,387 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [139]:
candidates_scored

['Is it a root vegetable? (90% - 10%)',
 'Is it commonly used in making purees? (70% - 30%)',
 'Is it commonly used in making baby food? (60% - 40%)',
 'Is it commonly used in making juices? (20% - 80%)',
 'Is it a tuber? (50% - 50%)',
 'Is it commonly used in making pies? (40% - 60%)',
 'Is it commonly used in making fries? (30% - 70%)',
 'Is it commonly used in making casseroles? (50% - 50%)',
 'Is it commonly used in making curries? (30% - 70%)',
 'Is it commonly used in stews? (80% - 20%)']

In [140]:
reasoning, question = await pick_question(client, game, candidates_scored)

2024-07-05 21:09:11,411 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [141]:
reasoning

'We know that the object is an edible, non-fruit, non-animal, non-dairy vegetable that is not commonly green or red, is commonly cooked before eating, and is commonly used in soups and is orange in color. Given these constraints, the vegetable is likely to be something like a carrot or a pumpkin. To narrow it down further, we should ask a question that splits the remaining possibilities effectively. The question "Is it a root vegetable?" is highly informative because it will help us distinguish between root vegetables like carrots and non-root vegetables like pumpkins. '

In [142]:
question

'Is it a root vegetable?'

In [143]:
reasoning, guessed_keyword = await guess(client, game)

2024-07-05 21:09:14,441 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [144]:
print(reasoning)

The keyword is a vegetable that is not green or red, is commonly cooked before eating, and is often used in soups. It is also commonly orange in color. Given these characteristics, and considering that both pumpkin and carrot have been guessed already, another common orange vegetable that fits these criteria is sweet potato. 


In [145]:
guessed_keyword

'sweet potato'

In [153]:
reasoning, answer_str = await answer(client, game, "carrot", "Is it commonly used in sex?")

2024-07-05 21:28:51,702 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [154]:
print(reasoning)

Carrots are not commonly associated with sexual use. While there might be rare instances or jokes about it, it is not a common or typical use for carrots. 


In [156]:
print(answer_str)

no
